### Import and installations

In [ ]:
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install flair

In [ ]:
import bz2
import json
import os.path
import pandas as pd
import time
from flair.models import TextClassifier
from flair.data import Sentence

In [ ]:
path_files = "/content/drive/MyDrive/"
path_us = path_files + "us_data/Filtered data/"
path_out = path_files + "data/"

pref = "quotes-"
suff = ".json.bz2"
data_names = []
data_names_speak = []
data_names_speak_sent = []

data_names_s = []
data_names_speak_s = []
data_names_speak_sent_s = []
year = {}
#Create the names to get the data and to store the data
#Note : 2 datasets were created for each year, 
# for that reason there are the variables data_names* and data_names_speak*
for (index, i) in enumerate(range(2015, 2021)):
  year[i] = index
  s = pref + str(i) + "-us"
  data_names.append(s + suff)
  data_names_speak.append(s + "_speakers_" + str(i) + suff)
  data_names_speak_sent.append(s + "_speakers_" + "_sent_" + suff)

  data_names_s.append(s  + "_sample"+ suff)
  data_names_speak_s.append(s + "_speakers_" + str(i) + "_sample"+  suff)
  data_names_speak_sent_s.append(s + "_speakers_" + "_sent_" + "_sample"+ suff)


print(data_names)
print(data_names_speak)

['quotes-2015-us.json.bz2', 'quotes-2016-us.json.bz2', 'quotes-2017-us.json.bz2', 'quotes-2018-us.json.bz2', 'quotes-2019-us.json.bz2', 'quotes-2020-us.json.bz2']
['quotes-2015-us_speakers_2015.json.bz2', 'quotes-2016-us_speakers_2016.json.bz2', 'quotes-2017-us_speakers_2017.json.bz2', 'quotes-2018-us_speakers_2018.json.bz2', 'quotes-2019-us_speakers_2019.json.bz2', 'quotes-2020-us_speakers_2020.json.bz2']


### Tools

In [ ]:
def sia_flair():
    start_time = time.time()
    print("Loading flair model...")
    sia = TextClassifier.load('sentiment-fast')
    print("Loading model done in  %s seconds." % (time.time() - start_time))
    return sia

def pred_flair(x, sia, batch_size):
    x_np = x.to_numpy()
    sentences = list(np.vectorize(lambda a : Sentence(a))(x_np))
    sia.predict(sentences,return_probabilities_for_all_classes=True, mini_batch_size=batch_size)
    scores = [ -1 * sent.labels[1].score + sent.labels[0].score for sent in sentences]

    return np.array(scores)

### Preprocess

In [ ]:
sia = sia_flair()

Loading flair model...
2021-12-10 15:33:18,944 https://nlp.informatik.hu-berlin.de/resources/models/sentiment-curated-fasttext-rnn/sentiment-en-mix-ft-rnn_v8.pt not found in cache, downloading to /tmp/tmp_8qh1wwc


100%|██████████| 1241977025/1241977025 [01:39<00:00, 12476252.13B/s]

2021-12-10 15:34:58,878 copying /tmp/tmp_8qh1wwc to cache at /root/.flair/models/sentiment-en-mix-ft-rnn_v8.pt


2021-12-10 15:35:03,285 removing temp file /tmp/tmp_8qh1wwc
2021-12-10 15:35:03,463 loading file /root/.flair/models/sentiment-en-mix-ft-rnn_v8.pt
Loading model done in  127.94758987426758 seconds.


### Process

In [ ]:
'''
get the sentiment chunk by chunk given a dataframe reader
'''
def reader_get_sentiment(df_r):
    df_s = []
    for chunk in df_r:
      df = pd.DataFrame()
      df['quoteId'] = chunk.quoteID
      df['sentiment'] = pred_flair_2(chunk.quotation, sia, batch_size=32)
      df_s.append(df)

    return pd.concat(df_s)

In [ ]:
for i in range(2015, 2021):
  df_reader = pd.read_json(path_us + data_names[year[i]], lines=True, compression='bz2', chunksize=10000)
  df_sent = reader_get_sentiment(df_reader)
  df_sent.to_pickle(path_us + "sent_" + str(i) + ".pkl")
  df_sent = None
  df_reader = None

  df_reader = pd.read_json(path_us + data_names_speak[year[i]], lines=True, compression='bz2', chunksize=10000)
  df_sent = reader_get_sentiment(df_reader)
  df_sent.to_pickle(path_us + "sent_speaker" + str(i) + ".pkl")
  df_sent = None
  df_reader = None
